In [1]:
from preprocessing.prep import prepare_paths, prepare_labels, remover_sublistas_redundantes, split_dataset

2024-05-12 20:11:58.540927: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-12 20:11:59.203207: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from dataset.dataset import __split_data__

In [3]:

from utils.dir import create_dir, __load_json__
from dataset.labels import __create_labels__, get_labels_name, split_label, get_all_structure, convert_label_to_string
from dataset.dataset import select_dataset, generate_tf_record, create_metadata, load_features


In [4]:
import pandas as pd

In [5]:
import os
import json
from tqdm import tqdm

In [6]:

args = pd.Series({
    "root_dir": "/mnt/disks/data/",
    "dataset_path": "/mnt/disks/data/fma/fma_large", 
    "embeddings": "music_style",
    "sequence_size": 1280,
    "train_id": "hierarchical_tworoots_dev",
    'sample_size':1
})



In [7]:
args, tracks_df = prepare_paths(args)

In [8]:
# Cria um dicionário que associa o ID de cada música aos IDs de seus gêneros musicais
#tracks_df = pd.read_csv(os.path.join(args.metadata_path,'tracks.csv'), header=[0,1], index_col=0)

In [9]:
tracks_df.track_genre_top.value_counts()

track_genre_top
Rock                   14172
Experimental           10603
Electronic              9358
Hip-Hop                 3546
Folk                    2801
Pop                     2331
Instrumental            2077
International           1375
Classical               1230
Jazz                     571
Old-Time / Historic      549
Spoken                   421
Country                  194
Soul-RnB                 175
Blues                    110
Easy Listening            24
Name: count, dtype: int64

In [10]:
tracks_df = tracks_df[tracks_df['track_genre_top'].isin(['Rock','Electronic'])]

In [11]:
#tracks_df.track.genres_all

In [12]:
tracks_df.columns

Index(['track_genres_all', 'track_id', 'file_path', 'index',
       'track_genre_top'],
      dtype='object')

In [13]:
def prepare_labels(tracks_df,args):
    ##### Labels
    # Loand genres df
    genres_df = pd.read_csv(os.path.join(args.metadata_path, 'genres.csv'))
    # Mapear os identificadores numéricos de gêneros para os nomes dos gêneros
    genre_names = dict(zip(genres_df['genre_id'], genres_df['title']))

    # Inicialize uma lista para armazenar todos os caminhos de gêneros para cada exemplo
    estruturas = []

    # Iterar sobre as faixas e seus gêneros associados
    for track_genres in tracks_df['track_genres_all']:
        #caminho_name = [genre_names[genre_id] for genre_id in track_genres]
        caminho_id = [get_all_structure(genre_id, genres_df) for genre_id in track_genres]
        estruturas.append(caminho_id)

    for idx, caminho in enumerate(estruturas):
        caminho.sort(key=len, reverse=True)
        estruturas[idx] = remover_sublistas_redundantes(caminho)

    ## Get structure form hierarchical classification
    #print(estruturas)
    tracks_df['y_true'] = estruturas
    tracks_df = tracks_df[['track_id', 'y_true']]

    ## Calculate labels_size
    max_depth = tracks_df.y_true.apply(lambda x: max([len(value) for value in x]))
    max_depth = int(max_depth.max())
    args['max_depth'] = max_depth
    print(f'max depth: {max_depth}')
    
    labels_name = []
    for level in range(1, max_depth+1):
        labels_name.append(f'label_{level}')
    tqdm.pandas()
    ## Gnetare categories_df
    #labels =  tracks_df.full_genre_id.progress_apply(lambda x: get_labels_per_level(x))
  
    #all_levels = categories_df.label5.progress_apply(lambda x: split_label(x))
    all_labels = []
    for idx, row in tqdm(enumerate(tracks_df.y_true)):
        for labels in row:
            labels.extend([""] * (max_depth - len(labels)))
            all_labels.append(labels)
            
    categories_df = pd.DataFrame(all_labels, columns=labels_name)

    categories_df.drop_duplicates(inplace=True)

    categories_df[f'label_{max_depth}_name'] = [get_labels_name(categorie, genres_df) for categorie in categories_df.values]

    data = __create_labels__(categories_df, max_depth)

    args['labels'] = data
    # Write labels file
    with open(args.categories_labels_path, 'w+') as f:
        f.write(json.dumps(data))

    return tracks_df, args


In [14]:
tracks_df, args = prepare_labels(tracks_df,args)

max depth: 4


23530it [00:00, 927826.46it/s]


In [15]:
#args.labels

In [16]:
tracks_df.y_true

17613                     [[12, , , ]]
10593                     [[15, , , ]]
98199                 [[12, 25, 89, ]]
42387                   [[12, 25, , ]]
100186            [[12, 25, 109, 361]]
                      ...             
30573                [[12, 25, 111, ]]
44246                   [[12, 27, , ]]
98568     [[12, 58, , ], [12, 31, , ]]
48899     [[12, 25, , ], [12, 85, , ]]
32676     [[12, 66, , ], [12, 85, , ]]
Name: y_true, Length: 23530, dtype: object

In [17]:
tracks_df.y_true.values.tolist()

[[[12, '', '', '']],
 [[15, '', '', '']],
 [[12, 25, 89, '']],
 [[12, 25, '', '']],
 [[12, 25, 109, 361]],
 [[12, 98, '', '']],
 [[15, 695, '', ''], [15, 236, '', ''], [15, 183, '', '']],
 [[12, '', '', '']],
 [[12, 85, '', '']],
 [[12, 58, '', '']],
 [[12, 26, '', '']],
 [[15, 297, '', ''], [15, 236, '', ''], [15, 183, '', '']],
 [[12, 66, '', '']],
 [[12, 88, '', '']],
 [[15, '', '', '']],
 [[15, '', '', '']],
 [[12, 25, 111, '']],
 [[12, '', '', '']],
 [[15, '', '', '']],
 [[15, '', '', '']],
 [[15, 297, '', '']],
 [[12, 25, '', '']],
 [[15, '', '', '']],
 [[12, 25, '', ''], [12, 85, '', '']],
 [[15, 495, '', ''], [15, 236, '', '']],
 [[12, 45, '', ''], [12, 31, '', '']],
 [[12, 25, 109, '']],
 [[15, 468, '', '']],
 [[15, 297, 240, '']],
 [[12, 58, '', '']],
 [[12, 25, 89, ''], [12, 25, 71, ''], [12, 45, 53, '']],
 [[15, 297, '', '']],
 [[12, 31, 439, '']],
 [[12, 25, 111, '']],
 [[15, 236, '', ''], [15, 181, '', '']],
 [[12, 25, '', '']],
 [[12, 26, '', '']],
 [[12, 25, '', ''], [1

In [18]:
max_depth = tracks_df.y_true.apply(lambda x: max([len(value) for value in x]))
max_depth = int(max_depth.max())

In [19]:
max_depth

4

In [20]:
args.labels

{'label_1': {12: 0, 15: 1},
 'label_1_name': {12: 'Rock', 15: 'Electronic'},
 'label_1_inverse': [12, 15],
 'label_1_count': 2,
 'label_2': {25: 1,
  98: 2,
  695: 3,
  236: 4,
  183: 5,
  85: 6,
  58: 7,
  26: 8,
  297: 9,
  66: 10,
  88: 11,
  495: 12,
  45: 13,
  31: 14,
  468: 15,
  181: 16,
  286: 17,
  27: 18,
  42: 19,
  296: 20,
  337: 21,
  359: 22,
  182: 23,
  184: 24,
  36: 25,
  185: 26,
  70: 27,
  314: 28,
  440: 29},
 'label_2_name': {25: 'Rock>Punk',
  98: 'Rock>Progressive',
  695: 'Electronic>Jungle',
  236: 'Electronic>IDM',
  183: 'Electronic>Glitch',
  85: 'Rock>Garage',
  58: 'Rock>Psych-Rock',
  26: 'Rock>Post-Rock',
  297: 'Electronic>Chip Music',
  66: 'Rock>Indie-Rock',
  88: 'Rock>New Wave',
  495: 'Electronic>Downtempo',
  45: 'Rock>Loud-Rock',
  31: 'Rock>Metal',
  468: 'Electronic>Dubstep',
  181: 'Electronic>Techno',
  286: 'Electronic>Trip-Hop',
  27: 'Rock>Lo-Fi',
  42: 'Electronic>Ambient Electronic',
  296: 'Electronic>Dance',
  337: 'Electronic>Drum

In [21]:
args

root_dir                                                   /mnt/disks/data/
dataset_path                                  /mnt/disks/data/fma/fma_large
embeddings                                                      music_style
sequence_size                                                          1280
train_id                                          hierarchical_tworoots_dev
sample_size                                                               1
job_path                  /mnt/disks/data/fma/trains/hierarchical_tworoo...
tfrecord_path             /mnt/disks/data/fma/trains/hierarchical_tworoo...
models_path                                          /mnt/disks/data/models
metadata_path                              /mnt/disks/data/fma/fma_metadata
metadata_train_path       /mnt/disks/data/fma/trains/hierarchical_tworoo...
categories_labels_path    /mnt/disks/data/fma/trains/hierarchical_tworoo...
max_depth                                                                 4
labels      

In [ ]:
split_dataset(tracks_df, args)

oversampling_size: 2


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 258/258 [00:00<00:00, 1866.21it/s]

Oversampling: [[12, 25, '', ''], [12, 26, '', '']]
Oversampling: [[12, 25, '', ''], [12, 314, '', '']]
Oversampling: [[12, 25, '', ''], [12, 359, '', '']]
Oversampling: [[12, 25, '', ''], [12, 45, '', ''], [12, 31, '', '']]
Oversampling: [[12, 25, '', ''], [12, 85, '', ''], [12, 45, '', '']]
Oversampling: [[12, 25, 64, ''], [12, 25, 71, '']]
Oversampling: [[12, 26, '', ''], [12, 66, '', ''], [12, 98, '', '']]
Oversampling: [[12, 26, 113, ''], [12, 25, 89, '']]
Oversampling: [[12, 27, '', ''], [12, 45, '', ''], [12, 31, '', '']]
Oversampling: [[12, 31, 167, ''], [12, 25, 89, '']]
Oversampling: [[12, 58, '', ''], [12, 27, '', ''], [12, 31, '', '']]
Oversampling: [[12, 58, '', ''], [12, 27, '', ''], [12, 66, '', '']]
Oversampling: [[12, 58, '', ''], [12, 27, '', ''], [12, 85, '', '']]
Oversampling: [[12, 58, '', ''], [12, 45, '', ''], [12, 31, '', '']]
Oversampling: [[12, 58, '', ''], [12, 45, '', ''], [12, 36, '', '']]
Oversampling: [[12, 66, '', ''], [12, 27, '', ''], [12, 85, '', '']]



2024-05-12 20:12:22.773395: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 667 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5
